In [1]:
import numpy as np
from matplotlib import pyplot
from scipy import linalg
import copy
import iMPS
from scipy.sparse import linalg
import funcs
import time

In [2]:
X = np.array([[0,1],[1,0]])
Id = np.eye(2)
X12 = np.zeros([1,1,64,64])
X12[0,0,:,:] = np.kron(X,np.kron(X,np.kron(Id,np.kron(Id,np.kron(Id,Id)))))

X13 = np.zeros([1,1,64,64])
X13[0,0,:,:] = np.kron(X,np.kron(Id,np.kron(X,np.kron(Id,np.kron(Id,Id)))))

X14 = np.zeros([1,1,64,64])
X14[0,0,:,:] = np.kron(X,np.kron(Id,np.kron(Id,np.kron(X,np.kron(Id,Id)))))

X16 = np.zeros([1,1,64,64])
X16[0,0,:,:] = np.kron(X,np.kron(Id,np.kron(Id,np.kron(Id,np.kron(Id,X)))))

X15 = np.zeros([1,1,64,64])
X15[0,0,:,:] = np.kron(X,np.kron(Id,np.kron(Id,np.kron(Id,np.kron(X,Id)))))


MPO16 = iMPS.iMPO()
MPO16.construct_from_tensor_list([X16,X16])

MPO14 = iMPS.iMPO()
MPO14.construct_from_tensor_list([X14,X14])

MPO13 = iMPS.iMPO()
MPO13.construct_from_tensor_list([X13,X13])

MPO12 = iMPS.iMPO()
MPO12.construct_from_tensor_list([X12,X12])

MPO15 = iMPS.iMPO()
MPO15.construct_from_tensor_list([X15,X15])


In [ ]:
lam1ss=[]
lam12ss=[]
lam13ss=[]
lam14ss=[]
lam15ss=[]
lam16ss=[]
dims=[]

Z = np.array([[1,0],[0,-1]])
X = np.array([[0,1],[1,0]])
A = np.array([[0.3,0.2],[-0.2,0.1]])
B = np.zeros([1,1,64])
B[0,0] =linalg.expm((np.kron(A,np.kron(A,A)))).reshape([64,])

MPS = iMPS.iMPS()
MPS.svd_threshold =1e-10
MPS.max_bond = 10
MPS.construct_from_tensor_list([B,B])

#rng = np.arange(0.,.01,0.04)
for p1 in np.arange(0.2,.5,0.03):   
    
    lam12s=[]
    lam13s=[]
    lam14s=[]
    lam1234s=[]
    lam15s=[]
    lam16s=[]
    for p3 in np.arange(0.3,.5,0.03):
        t1 = time.time()
        trans = funcs.single_trans_3layers(p1=p1,channel1='x_flip',p2=0,channel2='x_flip',p3=p3,channel3='x_flip')
        MPO = iMPS.iMPO()
        MPO.construct_from_tensor_list([trans,trans])

        MPS_power2 = iMPS.MPS_power_method_twosite(MPS,MPO,max_bond=10)
        MPS_power2.update(20,1e-3,1e-3)  
        
        dims.append(MPS_power2.MPS2.chi[0])
        
        strap = iMPS.strap(MPS_power2.MPS2,MPO12,MPS_power2.MPS2)
        lam12 = strap.calculate_eig()
        
        strap = iMPS.strap(MPS_power2.MPS2,MPO13,MPS_power2.MPS2)
        lam13 = strap.calculate_eig()
        
    
        strap = iMPS.strap(MPS_power2.MPS2,MPO14,MPS_power2.MPS2)
        lam14 = strap.calculate_eig()
        
        strap = iMPS.strap(MPS_power2.MPS2,MPO15,MPS_power2.MPS2)
        lam15 = strap.calculate_eig()
        
    
        strap = iMPS.strap(MPS_power2.MPS2,MPO16,MPS_power2.MPS2)
        lam16 = strap.calculate_eig()
        
        
        
    
        lam12s.append(lam12)
        lam13s.append(lam13)
        lam14s.append(lam14)
        lam15s.append(lam15)
        lam16s.append(lam16)
        t2 =  time.time()
        print(p2)
        print(t2-t1)

    lam12ss.append(lam12s)
    lam13ss.append(lam13s)
    lam14ss.append(lam14s)
    lam15ss.append(lam15s)
    lam16ss.append(lam16s)
    print(p1)